In [1]:
import numpy as np
from scipy.linalg import kron
from IPython.display import Markdown as md

spin_up = np.array([[1, 0]]).T
spin_down = np.array([[0, 1]]).T
# bit[0] = |0>, bit[1] = |1>
bit = [spin_up, spin_down]

In [2]:
def basis(string='00010'):
    '''string: the qubits sequence'''
    res = np.array([[1]])
    # 从最后一位开始往前数，做直积
    for idx in string[::-1]:
        res = kron(bit[int(idx)], res)    
    return np.matrix(res)

In [3]:
len(basis('00100000001000011100'))

1048576

In [4]:
def hilbert_space(nbit=5):
    nspace = 2**nbit
    for i in range(nspace):
        #bin(7) = 0b100
        binary = bin(i)[2:]
        nzeros = nbit - len(binary)
        yield '0'*nzeros + binary 

In [5]:
for mi in hilbert_space(nbit=3):
    print(mi, end=',')

000,001,010,011,100,101,110,111,

In [6]:
def wave_func(coef=[], seqs=[]):
    '''返回由振幅和几个Qubit序列表示的叠加态波函数，
       sum_i coef_i |psi_i> '''
    res = 0
    for i, a in enumerate(coef):
        res += a * basis(seqs[i])
    return np.matrix(res)

In [7]:
coef = [1j/np.sqrt(3), np.sqrt(2/3)]
seqs = ['000', '100']
s = wave_func(coef, seqs)
s

matrix([[0.        +0.57735027j],
        [0.        +0.j        ],
        [0.        +0.j        ],
        [0.        +0.j        ],
        [0.81649658+0.j        ],
        [0.        +0.j        ],
        [0.        +0.j        ],
        [0.        +0.j        ]])

In [8]:
def project(wave_func, direction):
    '''<Psi | phi_i> to get the amplitude '''
    return wave_func.H * direction

In [9]:
def decompose(wave_func):
    '''将叠加态波函数分解'''
    nbit = int(np.log2(len(wave_func)))
    amplitudes = []
    direct_str = []
    for seq in hilbert_space(nbit):
        direct = basis(seq)
        amp = project(wave_func, direct).A1[0]
        if np.linalg.norm(amp) != 0:
            amplitudes.append(amp)
            direct_str.append(seq)
    return amplitudes, direct_str

In [10]:
def print_wf(wf):
    coef, seqs = decompose(wf)
    latex = ""
    for i, seq in enumerate(seqs):
        latex += r"%s$|%s\rangle$"%(coef[i], seq)
        if i != len(seqs) - 1:
            latex += "+"
    return md(latex)

In [11]:
print_wf(s)

-0.5773502691896258j$|000\rangle$+(0.816496580927726+0j)$|100\rangle$

In [12]:
# Define the one-qubit and 2-qubit operation gates 
I = np.matrix("1 0; 0 1")
X = np.matrix("0 1; 1 0")
Y = np.matrix("0 -1j; 1j 0")
Z = np.matrix("1 0; 0 -1")
H = np.matrix("1 1; 1 -1") / np.sqrt(2)

CNOT = np.matrix("1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0")

SWAP = np.matrix("1 0 0 0; 0 0 1 0; 0 1 0 0; 0 0 0 1")

gates = {'I':I,  'X':X, 'Y':Y, 'Z':Z, 'H':H, 'CNOT':CNOT, 'SWAP':SWAP}

In [13]:
print_wf(X * spin_up)

1$|1\rangle$

In [14]:
print_wf(X * spin_down)

1$|0\rangle$

In [15]:
print_wf(X * (0.6 * spin_up + 0.8 * spin_down))

0.8$|0\rangle$+0.6$|1\rangle$

In [16]:
print_wf(H * bit[0])

0.7071067811865475$|0\rangle$+0.7071067811865475$|1\rangle$

In [17]:
print_wf(H * bit[1])

0.7071067811865475$|0\rangle$+-0.7071067811865475$|1\rangle$

In [18]:
s001 = basis('001')
IIX = kron(I, kron(I, X))
print_wf(IIX * s001)

1$|000\rangle$

In [19]:
s00 = basis('00')
print_wf(CNOT * s00)

1$|00\rangle$

In [20]:
s10 = basis('10')
print_wf(CNOT * s10)

1$|11\rangle$

In [23]:
print_wf(SWAP * basis('10'))

1$|01\rangle$